## Lab assignment 02

### Neural Machine Translation in the wild
In the third homework you are supposed to get the best translation you can for the EN-RU translation task.

Basic approach using RNNs as encoder and decoder is implemented for you. 

Your ultimate task is to use the techniques we've covered, e.g.

* Optimization enhancements (e.g. learning rate decay)

* Transformer/CNN/<whatever you select> encoder (with or without positional encoding)

* attention/self-attention mechanism

* pretraining the language models (for decoder and encoder)

* or just fine-tunning BART/ELECTRA/... ;)

to improve the translation quality. 

__Please use at least three different approaches/models and compare them (translation quality/complexity/training and evaluation time).__

Write down some summary on your experiments and illustrate it with convergence plots/metrics and your thoughts. Just like you would approach a real problem.

In [1]:
# Thanks to YSDA NLP course team for the data
# (who thanks tilda and deephack teams for the data in their turn)

import os
path_do_data = 'data.txt'
if not os.path.exists(path_do_data):
    print("Dataset not found locally. Downloading from github.")
    !wget https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt -nc

In [ ]:
# Baseline solution BLEU score is quite low. Try to achieve at least __21__ BLEU on the test set. 
# The checkpoints are:

# * __21__ - minimal score to submit the homework, 30% of points

# * __25__ - good score, 70% of points

# * __27__ - excellent score, 100% of points

### Warning! The code below is deeeeeeeply deprecated and is is provided only as simple guide.
We suggest you to stick to most recent pipelines here, e.g. by Huggingface: 
* Example notebook: [link](https://github.com/huggingface/notebooks/blob/main/examples/translation.ipynb)
* Converting your own dataset to specific format: [link](https://discuss.huggingface.co/t/correct-way-to-create-a-dataset-from-a-csv-file/15686/15)

In [2]:
import numpy as np
import pandas as pd
import torch
import random
import matplotlib.pyplot as plt
import time

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from transformers.modeling_outputs import BaseModelOutput
from transformers import T5Model, T5Tokenizer, T5Config, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu
from IPython.display import clear_output

import wandb

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
wandb.login()

In [3]:
with open('data.txt', 'r') as f:
    texts = f.read()

texts = texts.split(sep='\n')
texts = [row.split('\t') for row in texts]
texts_en = [row[0] for row in texts if len(row) == 2]
texts_ru = [row[1] for row in texts if len(row) == 2]

print('Num texts:', len(texts_en), len(texts_ru))
print('En max len:', max([len(row) for row in texts_en]))
print('Ru max len:', max([len(row) for row in texts_ru]))

Num texts: 50000 50000
En max len: 518
Ru max len: 431


In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LEN = 518
DEVICE

device(type='cuda')

In [5]:
class TextDataset(Dataset):
    def __init__(self, texts_en, texts_ru):
        self.texts_en = texts_en
        self.texts_ru = texts_ru
        
    def __len__(self):
        return len(self.texts_en)
    
    def __getitem__(self, idx):
        return self.texts_en[idx], self.texts_ru[idx]

In [6]:
train_texts_en, test_texts_en, train_texts_ru, test_texts_ru = train_test_split(texts_en, texts_ru, test_size=0.05, random_state=42)

train_dataset = TextDataset(train_texts_en, train_texts_ru)
test_dataset = TextDataset(test_texts_en, test_texts_ru)

In [7]:
n_epochs = 10
batch_size = 16
log_each_n_iterations = 500
generate_n = 1

In [8]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size)
generate_loader = DataLoader(test_dataset, generate_n, shuffle=True)


enc_name = 'distilbert-base-multilingual-cased'
dec_name = 't5-small'
# dec_name = "cointegrated/rut5-base-multitask"

enc_tokenizer = AutoTokenizer.from_pretrained(enc_name)
encoder = AutoModel.from_pretrained(enc_name).to(DEVICE)

dec_tokenizer = AutoTokenizer.from_pretrained(dec_name)
decoder = AutoModelForSeq2SeqLM.from_pretrained(dec_name).to(DEVICE)
# dec_tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-multitask")
config = T5Config(vocab_size=dec_tokenizer.vocab_size, d_model=encoder.config.dim, decoder_start_token_id=0)
decoder = T5ForConditionalGeneration(config).to(DEVICE)

for p in decoder.encoder.parameters():
    p.requires_grad = False
for p in decoder.decoder.parameters():
    p.requires_grad = True


LR = 1e-5
optimizer = torch.optim.AdamW(list(encoder.parameters()) + list(decoder.parameters()), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def encode(texts):
    encoded_input = enc_tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors='pt')
    with torch.no_grad():
        model_output = encoder(**encoded_input.to(encoder.device))
        embeddings = model_output.last_hidden_state
    return embeddings


def decode(embeddings, max_length=MAX_LEN, repetition_penalty=3.0, **kwargs):
    with torch.no_grad():
        out = decoder.generate(
            encoder_outputs=BaseModelOutput(last_hidden_state=embeddings), 
            max_length=max_length, 
            repetition_penalty=repetition_penalty,
            **kwargs
        )
        return [dec_tokenizer.decode(tokens, skip_special_tokens=True) for tokens in out]

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="nlp-lab2",
    notes="baseline",
    name='01',
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "encoder": enc_name,
    "decoder": dec_name,
    "epochs": n_epochs,
    }
)

text_table = wandb.Table(columns=["epoch", "true", "predict"])

train_history = []
iters = 1

for i in range(1, n_epochs + 1):
    print(f'[EPOCH {i}]')
    tqdm_iterator = tqdm(train_loader)
#     tqdm_iterator = train_loader

    for text_en_batch, text_ru_batch in tqdm_iterator:
        encoder.train()
        decoder.train()
        x = enc_tokenizer(text_en_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)
        y = dec_tokenizer(text_ru_batch, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LEN).to(DEVICE)

        y.input_ids[y.input_ids == 0] = -100  # не учитываем паддинг
        embeds = encoder(**x.to(encoder.device))
        embeds = embeds.last_hidden_state.to(DEVICE)

        loss = decoder(
            encoder_outputs=BaseModelOutput(last_hidden_state=embeds),
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        wandb.log({"batch loss": loss.item()})
        train_history.append((iters, loss.item()))
        
        if iters % log_each_n_iterations == 0:
            encoder.eval()
            decoder.eval()
            
            en, ru = next(iter(generate_loader))
            embeds = encode(en)
            generated = decode(embeds, max_length=MAX_LEN, repetition_penalty=None)
            print(ru[0])
            print('\n\n'.join(generated))
            
            text_table.add_data(str(iters), ru[0], ''.join(generated))
            
        
        iters += 1
    scheduler.step()

wandb.log({"table": text_table})
wandb.finish()

wandb: Currently logged in as: naumenko-km. Use `wandb login --relogin` to force relogin


[EPOCH 1]


 17%|█▋        | 500/2968 [03:23<1:21:38,  1.98s/it]

За 4 минуты гости могут дойти от апартаментов Seascape Holidays Sea Breeze on Garrick до серф-клуба Port Douglas и торгового центра Marina Mirage.
артамента аааааартаментааааааааааа аааааав ав автав артаментав автав автартаментав автав артаментав автав артаментав автав артаментав автартаментав ав артаментав автавтартаментав ав артаментав автартаментав ав артаментав автартаментав ав артаментав артаментав автартаментав ав артаментав артаментав автартаментав ав артаментав артаментав автартаментав ав артаментав артаментав артаментав автартаментав ав артаментав артаментав автартаментав артаментав ав артаментав артаментав артаментав артамента


 34%|███▎      | 1000/2968 [06:45<58:20,  1.78s/it] 

...

100%|██████████| 2968/2968 [19:34<00:00,  2.53it/s]


[EPOCH 6]


  5%|▌         | 160/2968 [01:04<28:24,  1.65it/s]

Поездка до аэропорта Домодедово займет 50 минут.
оедка до ароорта уааартаментов оед ааааарта ааас артаментов.


 20%|█▉        | 592/2968 [03:54<16:08,  2.45it/s]

In [17]:
wandb.finish()

### Main part
__Here comes the preprocessing. Do not hesitate to use BPE or more complex preprocessing ;)__

Here are tokens from original (RU) corpus:

And from target (EN) corpus:

And here is example from train dataset:

Let's check the length distributions:

### Model side
__Here comes simple pipeline of NMT model learning. It almost copies the week03 practice__

__Let's take a look at our network quality__:

In [14]:
original_text = []
generated_text = []
encoder.eval()
decoder.eval()

for en, ru in tqdm(test_loader):
    embeds = encode(en)
    generated = decode(embeds, max_length=MAX_LEN, repetition_penalty=None)
    
    original_text.extend(ru)
    generated_text.extend(generated)

# original_text = flatten(original_text)
# generated_text = flatten(generated_text)

100%|██████████| 157/157 [06:07<00:00,  2.34s/it]


In [15]:
print(original_text[0])
print(generated_text[0])

В распоряжении гостей общая кухня и общая гостиная.
 исле удоств ои кун с оеденна она и ое.


In [16]:
corpus_bleu([[text] for text in original_text], generated_text) * 100

26.33523700057311